In [31]:
import os
import pickle
import shutil
import numpy as np
from immuneML.data_model.encoded_data.EncodedData import EncodedData
from DaskLogisticRegression import DaskLogisticRegression
from DaskLogisticRegressioncluster import DaskLogisticRegressioncluster
from immuneML.analysis.data_manipulation.NormalizationType import NormalizationType
from immuneML.caching.CacheType import CacheType
from immuneML.encodings.EncoderParams import EncoderParams
from immuneML.encodings.kmer_frequency.KmerFreqRepertoireEncoder import KmerFreqRepertoireEncoder
from immuneML.util.ReadsType import ReadsType
from immuneML.encodings.kmer_frequency.sequence_encoding.SequenceEncodingType import SequenceEncodingType
from immuneML.environment.Constants import Constants
from immuneML.environment.EnvironmentSettings import EnvironmentSettings
from immuneML.environment.Label import Label
from immuneML.environment.LabelConfiguration import LabelConfiguration
from immuneML.hyperparameter_optimization.HPSetting import HPSetting
from immuneML.simulation.dataset_generation.RandomDatasetGenerator import RandomDatasetGenerator
from immuneML.util.PathBuilder import PathBuilder
from immuneML.workflows.instructions.ml_model_application.MLApplicationInstruction import MLApplicationInstruction
from immuneML.hyperparameter_optimization.core.HPUtil import HPUtil
from immuneML.hyperparameter_optimization.config.SplitConfig import SplitConfig


In [18]:
# from immuneML.hyperparameter_optimization.config.ReportConfig import ReportConfig
# datasplits = {
#     "SequenceLengthDistribution": {
#         "batch_size": 3
#     }
# }
# models1 = {"coef": "Coefficients"}


In [19]:
# from immuneML.hyperparameter_optimization.config.SplitType import SplitType


In [20]:
# reportconfig1 = ReportConfig(
#     data_splits=datasplits, models=models1)


In [21]:
# split_config_assesment = SplitConfig(
#     split_strategy=SplitType.RANDOM, split_count=1, training_percentage=0.7, reports=reportconfig1)


In [32]:
path = EnvironmentSettings.tmp_test_path / "mlapplicationtest/"
PathBuilder.build(path)

dataset = RandomDatasetGenerator.generate_repertoire_dataset(
    50, {5: 1}, {5: 1}, {"l1": {1: 0.5, 2: 0.5}}, path / 'dataset/')
label = Label("l1", [1, 2])
label_config = LabelConfiguration([label])

In [23]:
# train_val_datasets, test_datasets = HPUtil.split_data(
#     dataset, split_config_assesment, path, label)


In [33]:
dataset

In [25]:
# train_val_datasets[0].__dict__

{'encoded_data': None,
 'identifier': 'b7a0fd6c-e28b-11ec-96b8-9cb6d0fe1bec',
 'name': 'b7a0fd6c-e28b-11ec-96b8-9cb6d0fe1bec',
 'labels': {'l1': [1, 2]},
 'metadata_file': WindowsPath('C:/Users/karth/Desktop/PhD projects/immuneML/immuneML/test/tmp/mlapplicationtest/split_1/datasets/train_metadata.csv'),
 'metadata_fields': None,
 'repertoire_ids': None,
 'repertoires': [<immuneML.data_model.repertoire.Repertoire.Repertoire at 0x23b5cf4fdf0>,
  <immuneML.data_model.repertoire.Repertoire.Repertoire at 0x23b5df96700>]}

In [34]:
from immuneML.ml_methods.util.Util import Util


In [35]:


ml_method = DaskLogisticRegressioncluster()

# ml_method = DaskLogisticRegression()
encoder = KmerFreqRepertoireEncoder(NormalizationType.RELATIVE_FREQUENCY, ReadsType.UNIQUE, SequenceEncodingType.CONTINUOUS_KMER, 3,
                                    scale_to_zero_mean=True, scale_to_unit_variance=True)


enc_dataset = encoder.encode(dataset, EncoderParams(
    result_path=path, label_config=label_config, filename="tmp_enc_dataset.pickle", pool_size=4))


c:\Users\karth\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\Users\karth\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [ ]:

# train_enc_dataset = encoder.encode(train_val_datasets[0], EncoderParams(
#     result_path=path, label_config=label_config, filename="tmp_enc_dataset.pickle", pool_size=4))


In [28]:
# test_enc_dataset = encoder.encode(test_datasets[0], EncoderParams(
#     result_path=path, label_config=label_config, filename="tmp_test_enc_dataset.pickle", pool_size=4))


c:\Users\karth\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
c:\Users\karth\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [36]:
ml_method.fit(enc_dataset.encoded_data, label)

hp_setting = HPSetting(encoder, {"normalization_type": "relative_frequency", "reads": "unique", "sequence_encoding": "continuous_kmer",
                                 "k": 3, "scale_to_zero_mean": True, "scale_to_unit_variance": True}, ml_method, {}, [], 'enc1', 'ml1')

PathBuilder.build(path / 'result/instr1/')


{'max_iter': 1000, 'solver': 'lbfgs'}
<bound method DaskMlMethod._get_model_filename of <DaskLogisticRegressioncluster.DaskLogisticRegressioncluster object at 0x0000023B5FC2E5B0>>
{'penalty': 'l2', 'dual': False, 'tol': 0.0001, 'C': 1.0, 'fit_intercept': True, 'intercept_scaling': 1.0, 'class_weight': None, 'random_state': None, 'solver': 'lbfgs', 'max_iter': 1000, 'multi_class': 'ovr', 'verbose': 0, 'warm_start': False, 'n_jobs': 1, 'solver_kwargs': None}
DASK workflow


c:\Users\karth\anaconda3\envs\test\lib\site-packages\dask\config.py:643: UserWarning: Configuration key "fuse_ave_width" has been deprecated. Please use "optimization.fuse.ave-width" instead
  warnings.warn(


WindowsPath('C:/Users/karth/Desktop/PhD projects/immuneML/immuneML/test/tmp/mlapplicationtest/result/instr1')

In [37]:
ml_app = MLApplicationInstruction(
    dataset, label_config, hp_setting, 4, "instr1")
ml_app.run(path / 'result/')

predictions_path = path / "result/instr1/predictions.csv"


2022-06-02 17:56:19.241092: Encoding started...
2022-06-02 17:56:25.143450: Encoding finished.


c:\Users\karth\anaconda3\envs\test\lib\site-packages\sklearn\utils\validation.py:593: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [38]:
ml_method

In [39]:
import pandas as pd

In [40]:
data_re = pd.read_csv(predictions_path)

In [41]:
data_re

,repertoire_file,example_id,l1,l1_1_proba,l1_2_proba
0,rep_0.npy,a9be255436cc4ab999763602fdb14ecf,2,0.003433,0.996567
1,rep_1.npy,9115936265f74ba5a6949d9027f8360e,1,0.989584,0.010416
2,rep_2.npy,2ad2f4f72ab94b96bceb93f156029fc5,1,0.989160,0.010840
3,rep_3.npy,d99e78e94ba7447ca9f9dc087e57d491,2,0.003787,0.996213
4,rep_4.npy,c008d6e332f24b3d9ec343d267975983,1,0.988669,0.011331
5,rep_5.npy,165f233453824b368220006c9aa495db,2,0.003426,0.996574
6,rep_6.npy,23e5d575ce3c4ec98e8701f26ad830ae,1,0.988694,0.011306
7,rep_7.npy,68908aa1a9624c0596c8a52d6a4fc0c5,2,0.004209,0.995791
8,rep_8.npy,a2bae21247af44debf4eed09b5967e96,2,0.003425,0.996575
9,rep_9.npy,93a244cc8a524ffe95ffad692c6d1096,1,0.989134,0.010866


In [11]:
import hashlib
import hmac
import secrets
from skl2onnx import to_onnx

from dotenv import load_dotenv
import os

load_dotenv()


True

In [31]:
ml_method.model

LogisticRegression(max_iter=1000, solver='saga')

In [47]:
def convert_to_onnx( model,training_data, path):

        if model is not None:

            onx = to_onnx(model, training_data[:10].astype(numpy.float32))

        PathBuilder.build(path)
        file_path = path / f"{type(model).__name__}.onnx"

        with open(file_path, "wb") as f:
            f.write(onx.SerializeToString())


In [48]:
onnx_path = path / "result"


In [22]:
ml_method

In [49]:
training_data = enc_dataset.encoded_data.examples


In [50]:
training_data[:4]

array([[-0.20412415, -0.14285714, -0.14285714, ..., -0.14285714,
        -0.14285714, -0.14285714],
       [-0.20412415, -0.14285714, -0.14285714, ..., -0.14285714,
        -0.14285714,  7.        ],
       [-0.20412415, -0.14285714, -0.14285714, ..., -0.14285714,
        -0.14285714, -0.14285714],
       [-0.20412415, -0.14285714, -0.14285714, ...,  7.        ,
        -0.14285714, -0.14285714]])

In [51]:
convert_to_onnx(ml_method.model, training_data , onnx_path)

In [37]:
onnx_path_model = onnx_path / "LogisticRegression.onnx"

In [52]:
onx = to_onnx(ml_method.model, training_data[:10].astype(numpy.float32))


In [54]:

import numpy
import onnxruntime as rt

sess = rt.InferenceSession(onx.SerializeToString())
input_name = sess.get_inputs()[0].name
pred_onx = sess.run(None, {input_name: training_data.astype(numpy.float32)})[0]
print(pred_onx)


[0 0 1 0 0 1 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 1 0
 0 0 1 0 1 1 0 1 0 0 0 1 0]


In [56]:
model = ml_method.model

In [57]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.environ.get('secret'))


7adf9e73165b41260977c4f16a22d3f587445df9d0495ee0417bf01b33307f50


In [59]:
import hashlib
import hmac
import dill


data = dill.dumps(model)
digest = hmac.new(str.encode(os.environ.get('secret')),
                  data, hashlib.blake2b).hexdigest()
print(digest)
print(type(digest))

path = PathBuilder.build("../ml_methods")

file_path = path / \
    f"{str(digest)}-{type(model).__name__}.pickle"

with file_path.open("wb") as file:
    dill.dump(model, file)


caed5b672400e0099bbd60c0b628b8c3d09faef380851fdcde67b9a2f41d97ba9a791f805bc8c0e65d4e2733310178d9b36d038f1445c866863b95fd6a31fbab
<class 'str'>


In [60]:
filename = file_path.name

with open(filename, "rb") as f:
    read_data = f.read()


print(filename.split('-')[0])

caed5b672400e0099bbd60c0b628b8c3d09faef380851fdcde67b9a2f41d97ba9a791f805bc8c0e65d4e2733310178d9b36d038f1445c866863b95fd6a31fbab


In [62]:
filename = file_path.name
digest, pickle_data = filename.split('-')

expected_digest = hmac.new(
    str.encode(os.environ.get('secret')), read_data, hashlib.blake2b).hexdigest()
print(expected_digest)
if not secrets.compare_digest(digest, expected_digest):
    raise SystemExit("Invalid Signature")
    exit(1)
else:
    obj = dill.loads(data)


caed5b672400e0099bbd60c0b628b8c3d09faef380851fdcde67b9a2f41d97ba9a791f805bc8c0e65d4e2733310178d9b36d038f1445c866863b95fd6a31fbab
